In [11]:
import pandas as pd

In [14]:
df_winning = pd.read_csv('Winning-members-(Friday-1720).csv')

In [ ]:
df_winning.head()   

,ons_id,constituency_name,region_name,country_name,constituency_type,time_recorded,result,party_abbreviation,party_name,title,firstname,surname,middlenames,mnis_id,gender,candidate_type,old_constituency
0,W07000081,Aberafan Maesteg,Wales,Wales,County,2024-07-05 04:07:00,Lab hold,Lab,Labour,NaN,Stephen,Kinnock,NaN,4359.0,Male,Re-elected Member,Aberavon
1,S14000060,Aberdeen North,Scotland,Scotland,Borough,2024-07-05 05:00:00,SNP hold,SNP,Scottish National Party,NaN,Kirsty,Blackman,Ann,4357.0,Female,Re-elected Member,Aberdeen North
2,S14000061,Aberdeen South,Scotland,Scotland,Borough,2024-07-05 05:02:00,SNP hold,SNP,Scottish National Party,NaN,Stephen,Flynn,Mark,4735.0,Male,Re-elected Member,Aberdeen South
3,S14000062,Aberdeenshire North and Moray East,Scotland,Scotland,County,2024-07-05 05:47:00,SNP gain from Con,SNP,Scottish National Party,NaN,Seamus,Logan,NaN,NaN,Male,New member,NaN
4,S14000063,Airdrie and Shotts,Scotland,Scotland,County,2024-07-05 05:04:00,Lab gain from SNP,Lab,Labour,NaN,Kenneth,Stevenson,NaN,NaN,Male,New member,NaN


In [15]:
df_defeated = pd.read_csv('Defeated-MPs-(Friday-1720).csv')

In [11]:
df_defeated.columns

Index(['ons_id', 'constituency_name', 'region_name', 'country_name',
       'constituency_type', 'time_recorded', 'result', 'party_abbreviation',
       'party_name', 'title', 'firstname', 'surname', 'middlenames', 'mnis_id',
       'gender', 'defeated_mp', 'former_mp'],
      dtype='object')

In [15]:
df_winning.columns

Index(['ons_id', 'constituency_name', 'region_name', 'country_name',
       'constituency_type', 'time_recorded', 'result', 'party_abbreviation',
       'party_name', 'title', 'firstname', 'surname', 'middlenames', 'mnis_id',
       'gender', 'candidate_type', 'old_constituency'],
      dtype='object')

In [16]:
df_winning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ons_id              649 non-null    object        
 1   constituency_name   649 non-null    object        
 2   region_name         649 non-null    object        
 3   country_name        649 non-null    object        
 4   constituency_type   649 non-null    object        
 5   time_recorded       649 non-null    datetime64[ns]
 6   result              649 non-null    object        
 7   party_abbreviation  649 non-null    object        
 8   party_name          649 non-null    object        
 9   title               70 non-null     object        
 10  firstname           649 non-null    object        
 11  surname             649 non-null    object        
 12  middlenames         236 non-null    object        
 13  mnis_id             315 non-null    float64       

In [17]:
df_defeated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ons_id              217 non-null    object        
 1   constituency_name   217 non-null    object        
 2   region_name         217 non-null    object        
 3   country_name        217 non-null    object        
 4   constituency_type   217 non-null    object        
 5   time_recorded       217 non-null    datetime64[ns]
 6   result              217 non-null    object        
 7   party_abbreviation  217 non-null    object        
 8   party_name          217 non-null    object        
 9   title               32 non-null     object        
 10  firstname           217 non-null    object        
 11  surname             217 non-null    object        
 12  middlenames         123 non-null    object        
 13  mnis_id             217 non-null    int64         

In [ ]:
df

In [28]:
import requests
from bs4 import BeautifulSoup

url = 'https://election.news.sky.com/elections/general-election-2024/aberafan-maesteg-1'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

constituencies = soup.find_all('div', class_='styles_page-header-title__5Rknt display-any')

for constituency in constituencies:
    title = constituency.get_text()
    result_tag = constituency.find_next('div', class_='styles_banner__zyX_k ui-glass-swoosh display-any')
    result = result_tag.get_text() if result_tag else 'No result'
    print(f"Constituency: {title}, Result: {result}")


In [30]:
constituencies

[]

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://election.news.sky.com/elections/general-election-2024/'

constituencies = ['aberafan-maesteg-1']

data = []

for constituency in constituencies:
    url = base_url + constituency
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Check and find constituency name
    constituency_name_tag = soup.find('h1', class_='constituency-name')
    if constituency_name_tag:
        constituency_name = constituency_name_tag.text.strip()
    else:
        print(f"Constituency name not found for URL: {url}")
        continue
    
    # Check and find candidates
    candidates = soup.find_all('div', class_='candidate-result')
    if not candidates:
        print(f"No candidates found for URL: {url}")
        continue
    
    for candidate in candidates:
        party_tag = candidate.find('span', class_='party-name')
        votes_tag = candidate.find('span', class_='vote-total')
        percentage_tag = candidate.find('span', class_='vote-percentage')
        
        if party_tag and votes_tag and percentage_tag:
            party = party_tag.text.strip()
            votes = votes_tag.text.strip()
            percentage = percentage_tag.text.strip()
            data.append([constituency_name, party, votes, percentage])
        else:
            print(f"Missing data for candidate in constituency: {constituency_name}")

df = pd.DataFrame(data, columns=['Constituency', 'Party', 'Votes', 'Percentage'])
df.head()

Constituency name not found for URL: https://election.news.sky.com/elections/general-election-2024/aberafan-maesteg-1


,Constituency,Party,Votes,Percentage


In [3]:
data = data  = {
  "props": {
    "pageProps": {
      "components": {
        "consentManagement": {
          "enabled": True,
          "accountId": 630,
          "propertyId": 9804,
          "privacyManagerId": 617780,
          "spv2": True,
          "propertyHref": "https://election.news.sky.com/",
          "joinHref": "False"
        },
        "chartbeat": {
          "enabled": "True",
          "uid": "34974",
          "domain": "news.sky.com",
          "pageTitle": "Aberafan\r\n        Maesteg | General Election 2024"
        },
        "analytics": {
          "applicationEnvironment": "production",
          "analyticsEnvironment": "prod",
          "application": "Elections",
          "name": "Aberafan Maesteg",
          "breadcrumb": [
            "Election",
            "General Election 2024",
            "Area",
            "Aberafan Maesteg"
          ],
          "type": "area",
          "tags": {
            "name": "section",
            "value": "Election:Area"
          },
          "pageTitle": "Aberafan\r\n        Maesteg | General Election 2024"
        },
        "launch": {
          "appId": "ENdec3197734dd4b629a9283183faf094b"
        },
        "electionNav": {
          "navigation": [
            {
              "label": "Vote 2022",
              "link": "/"
            },
            {
              "label": "Northern Ireland Assembly",
              "link": "/elections/northern-ireland-2022",
              "current": True
            }
          ]
        },
        "footerData": {
          "properties": {
            "logo": "/resources/sky-spectrum-logo.png",
            "logo-alt": "Sky\r\n        logo",
            "logo-link": "//www.sky.com",
            "logo-height": "46px",
            "logo-width": "72px",
            "copyright-text": "© 2024 Sky UK"
          },
          "navigation": {
            "items": [
              {
                "label": "About Sky News",
                "items": [
                  {
                    "label": "About Us",
                    "link": "https://news.sky.com/info/policies-and-standards"
                  },
                  {
                    "label": "Sky\r\n        News Profiles",
                    "link": "https://news.sky.com/sky-news-profiles"
                  },
                  {
                    "label": "Sky News International",
                    "link": "http://www.skynewsinternational.com",
                    "external": True
                  },
                  {
                    "label": "Sky News Library Sales",
                    "link": "https://news.sky.com/info/library-sales"
                  },
                  {
                    "label": "Site\r\n        Map",
                    "link": "https://news.sky.com/info/sitemap"
                  },
                  {
                    "label": "Editorial Guidelines",
                    "link": "https://components.news.sky.com/files/editorial_guidelines.pdf",
                    "external": True
                  },
                  {
                    "label": "Sky News Board",
                    "link": "https://components.news.sky.com/files/terms_of_reference_of_the_sky_news_board.pdf",
                    "external": True
                  }
                ]
              },
              {
                "label": "Sky\r\n        News Services",
                "items": [
                  {
                    "label": "Sky News RSS",
                    "link": "https://news.sky.com/info/rss"
                  },
                  {
                    "label": "Sky News For Your Phone",
                    "link": "https://news.sky.com/info/smartphones"
                  },
                  {
                    "label": "Sky News Radio",
                    "link": "https://news.sky.com/info/radio"
                  }
                ]
              },
              {
                "label": "Sky\r\n        Channels",
                "items": [
                  {
                    "label": "Sky Witness",
                    "link": "https://www.sky.com/watch/channel/sky-witness",
                    "external": True
                  },
                  {
                    "label": "Sky Atlantic",
                    "link": "https://www.sky.com/watch/channel/sky-atlantic",
                    "external": True
                  },
                  {
                    "label": "Sky Arts",
                    "link": "https://www.sky.com/watch/channel/sky-arts",
                    "external": True
                  },
                  {
                    "label": "Sky\r\n        Cinema",
                    "link": "https://www.sky.com/watch/channel/sky-cinema",
                    "external": True
                  },
                  {
                    "label": "Sky Sports",
                    "link": "https://www.skysports.com",
                    "external": True
                  }
                ]
              },
              {
                "label": "More Sky Sites",
                "items": [
                  {
                    "label": "NOW",
                    "link": "https://www.nowtv.com",
                    "external": True
                  },
                  {
                    "label": "Sky\r\n        Zero",
                    "link": "https://www.skyzero.sky",
                    "external": True
                  },
                  {
                    "label": "Sky Academy Studios",
                    "link": "https://www.skyup.sky/academy-studios",
                    "external": True
                  },
                  {
                    "label": "Bigger Picture",
                    "link": "https://www.skygroup.sky/corporate/bigger-picture",
                    "external": True
                  },
                  {
                    "label": "Sky\r\n        Group",
                    "link": "https://www.skygroup.sky",
                    "external": True
                  },
                  {
                    "label": "Sky Bet",
                    "link": "https://m.skybet.com",
                    "external": True
                  },
                  {
                    "label": "Sky.com",
                    "link": "https://www.sky.com",
                    "external": True
                  },
                  {
                    "label": "Sky News Arabia",
                    "link": "https://www.skynewsarabia.com",
                    "external": True
                  },
                  {
                    "label": "Advertise\r\n        With Us",
                    "link": "https://www.skymedia.co.uk/?utm_source=Sky%20News&amp;utm_medium=Web%20page&amp;utm_content=Sky%20News%20web%20page&amp;utm_campaign=Sky%20News%20web%20page",
                    "external": True
                  }
                ]
              }
            ]
          },
          "social": {
            "items": [
              {
                "id": "facebook",
                "label": "Facebook",
                "link": "https://www.facebook.com/skynews",
                "external": True,
                "icon": "<svg\r\n        xmlns='http://www.w3.org/2000/svg' role='img' aria-hidden='True' viewBox='0 0 384 384'><path d='M283.232 207.471l8.248-64.043h-63.428v-40.888c0-18.549 5.187-31.183 31.758-31.183l33.896-.021v-57.272c-5.846-.784-25.956-2.531-49.408-2.531-48.9\r\n        0-82.357 29.861-82.357 84.668v47.227h-55.311v64.043h55.312v164.332h66.113v-164.332h55.177z'/></svg>"
              },
              {
                "id": "x",
                "label": "X",
                "link": "https://x.com/SkyNews",
                "external": True,
                "icon": "<svg xmlns='http://www.w3.org/2000/svg' viewBox='0\r\n        0 1200 1227'><path d='M714.163 519.284L1160.89 0H1055.03L667.137 450.887L357.328 0H0L468.492 681.821L0 1226.37H105.866L515.491 750.218L842.672 1226.37H1200L714.137 519.284H714.163ZM569.165 687.828L521.697 619.934L144.011 79.6944H306.615L611.412\r\n        515.685L658.88 583.579L1055.08 1150.3H892.476L569.165 687.854V687.828Z'/></svg>"
              },
              {
                "id": "youtube",
                "label": "YouTube",
                "link": "https://www.youtube.com/user/skynews",
                "external": True,
                "icon": "<svg xmlns='http://www.w3.org/2000/svg'\r\n        role='img' aria-hidden='True' viewBox='0 0 384 384'><path d='M368.449,119.707c0,0-3.527-24.891-14.355-35.848c-13.734-14.379-29.121-14.449-36.188-15.293c-50.531-3.656-126.328-3.656-126.328-3.656h-0.152c0,0-75.809,0-126.34,3.656c-7.055,0.844-22.441,0.914-36.188,15.293c-10.816,10.957-14.344,35.848-14.344,35.848s-3.609,29.215-3.609,58.441v27.398c0,29.215,3.609,58.441,3.609,58.441s3.527,24.891,14.344,35.848c13.746,14.379,31.781,13.922,39.82,15.434c28.887,2.766,122.777,3.633,122.777,3.633s75.879-0.117,126.41-3.773c7.066-0.844,22.453-0.914,36.188-15.293c10.828-10.957,14.355-35.848,14.355-35.848s3.609-29.227,3.609-58.441v-27.398C372.059,148.922,368.449,119.707,368.449,119.707\r\n        M154.219,238.746l-0.023-101.461l97.582,50.906L154.219,238.746z'/></svg>"
              },
              {
                "id": "instagram",
                "label": "Instagram",
                "link": "https://www.instagram.com/skynews/",
                "external": True,
                "icon": "<svg xmlns='http://www.w3.org/2000/svg' role='img'\r\n        aria-hidden='True' viewBox='0 0 512 512'><linearGradient id='instagram-gradient' x1='100%' x2='0%' y1='3.3961%' y2='100%'><stop stop-color='#BB00B6 offset='0'/><stop stop-color='#CE007C' offset='.22779'/><stop\r\n        stop-color='#F20000' offset='.48415'/><stop stop-color='#FD6200' offset='.70142'/><stop stop-color='#FFD700' offset='1'/></linearGradient><path d='M256 46.1c68.4 0 76.5.3 103.4 1.5 25 1.1 38.5 5.3 47.5 8.8\r\n        11.9 4.6 20.5 10.2 29.4 19.1 9 9 14.5 17.5 19.2 29.4 3.5 9 7.7 22.6 8.8 47.5 1.2 27 1.5 35.1 1.5 103.4s-.3 76.5-1.5 103.4c-1.1 25-5.3 38.5-8.8 47.5-4.6 11.9-10.2 20.5-19.1 29.4-9 9-17.5 14.5-29.4 19.2-9 3.5-22.6 7.7-47.5 8.8-27 1.2-35.1 1.5-103.4\r\n        1.5s-76.5-.3-103.4-1.5c-25-1.1-38.5-5.3-47.5-8.8-11.9-4.6-20.5-10.2-29.4-19.1-9-9-14.5-17.5-19.2-29.4-3.5-9-7.7-22.6-8.8-47.5-1.2-27-1.5-35.1-1.5-103.4s.3-76.5 1.5-103.4c1.1-25 5.3-38.5 8.8-47.5 4.6-11.9 10.2-20.5 19.1-29.4 9-9 17.5-14.5 29.4-19.2\r\n        9-3.5 22.6-7.7 47.5-8.8 26.9-1.2 35-1.5 103.4-1.5M256 0c-69.5 0-78.2.3-105.5 1.5-27.2 1.2-45.9 5.6-62.1 11.9C71.5 20 57.2 28.7 43 43 28.7 57.2 20 71.5 13.4 88.3c-6.3 16.3-10.7 34.9-11.9 62.1C.3 177.8 0 186.5 0 256s.3 78.2 1.5 105.5c1.2 27.2 5.6\r\n        45.9 11.9 62.1C20 440.5 28.7 454.8 43 469c14.2 14.2 28.5 23 45.3 29.5 16.3 6.3 34.9 10.7 62.1 11.9 27.3 1.2 36 1.5 105.5 1.5s78.2-.3 105.5-1.5c27.2-1.2 45.9-5.6 62.1-11.9 16.8-6.5 31.1-15.3 45.3-29.5 14.2-14.2 23-28.5 29.5-45.3 6.3-16.3 10.7-34.9\r\n        11.9-62.1 1.2-27.3 1.5-36 1.5-105.5s-.3-78.2-1.5-105.5c-1.2-27.2-5.6-45.9-11.9-62.1-6.3-17-15-31.3-29.3-45.5-14.2-14.2-28.5-23-45.3-29.5-16.3-6.3-34.9-10.7-62.1-11.9C334.2.3 325.5 0 256 0z'/><path d='M256 124.5c-72.6 0-131.5 58.9-131.5\r\n        131.5S183.4 387.5 256 387.5 387.5 328.6 387.5 256 328.6 124.5 256 124.5zm0 216.8c-47.1 0-85.3-38.2-85.3-85.3 0-47.1 38.2-85.3 85.3-85.3 47.1 0 85.3 38.2 85.3 85.3 0 47.1-38.2 85.3-85.3 85.3z'/><circle cx='392.7' cy='119.3' r='30.7'/></svg>"
              }
            ]
          },
          "supplementary": {
            "items": [
              {
                "label": "Terms\r\n        & Conditions",
                "link": "https://www.sky.com/help/articles/skycom-terms-and-conditions",
                "external": True
              },
              {
                "label": "Privacy & Cookies",
                "link": "https://www.sky.com/help/articles/sky-privacy-and-cookies-notice",
                "external": True
              },
              {
                "label": "Privacy\r\n        Options",
                "link": "#privacy-options"
              },
              {
                "label": "Accessibility",
                "link": "https://www.skyaccessibility.sky",
                "external": True
              },
              {
                "label": "Contact Us",
                "link": "https://news.sky.com/info/contact-us",
                "external": False
              }
            ]
          },
          "icons": {
            "down": "<svg width='24'\r\n        height='14' viewBox='0 0 24 14' xmlns='http://www.w3.org/2000/svg'><title>Down</title><path d='M0 1.29c0 .12.022.227.067.323.044.097.09.167.133.21l11.08 11.06c.193.222.442.333.746.333s.552-.11.745-.333L23.74 1.936c.104-.104.167-.193.19-.267.022-.075.03-.164.022-.268-.008-.104-.056-.215-.145-.334l-.29-.423C23.31.408\r\n        23.096.23 22.874.11c-.06 0-.09-.014-.09-.043L22.762 0 11.927 10.858 1.068 0l-.11.11C.852.216.744.305.633.38.524.452.414.54.31.645.253.75.186.857.11.968c-.073.11-.11.22-.11.322z' /></svg>"
          }
        },
        "headerData": {
          "properties": {
            "modifier": "news",
            "logo": "/resources/sky-news-logo.svg?v=1",
            "logo-dark-mode": "/resources/sky-news-logo-dark.svg?v=1&bypass-service-worker",
            "logo-alt": "sky\r\n        news logo",
            "logo-link": "https://news.sky.com/",
            "logo-a11y-text": "Sky News - back to home"
          },
          "amp": {
            "navigation": {
              "items": [
                {
                  "label": "Home",
                  "link": "https://news.sky.com/",
                  "key": "home"
                },
                {
                  "label": "Election",
                  "link": "https://news.sky.com/politics",
                  "key": "politics",
                  "current": True
                },
                {
                  "label": "UK",
                  "link": "https://news.sky.com/uk",
                  "key": "uk"
                },
                {
                  "label": "World",
                  "link": "https://news.sky.com/world",
                  "key": "world"
                },
                {
                  "label": "Israel-Hamas\r\n        War",
                  "link": "https://news.sky.com/israel-hamas-war",
                  "key": "israel-hamas-war"
                },
                {
                  "label": "US",
                  "link": "https://news.sky.com/us",
                  "key": "us"
                },
                {
                  "label": "Climate",
                  "link": "https://news.sky.com/climate",
                  "key": "climate"
                },
                {
                  "label": "Science & Tech",
                  "link": "https://news.sky.com/technology",
                  "key": "tech"
                },
                {
                  "label": "Business",
                  "link": "https://news.sky.com/business",
                  "key": "business"
                },
                {
                  "label": "Ents\r\n        & Arts",
                  "link": "https://news.sky.com/entertainment",
                  "key": "entertainment"
                },
                {
                  "label": "Offbeat",
                  "link": "https://news.sky.com/strangenews",
                  "key": "strangenews"
                },
                {
                  "label": "Travel",
                  "link": "https://news.sky.com/travel",
                  "key": "travel"
                },
                {
                  "label": "Analysis",
                  "link": "https://news.sky.com/analysis",
                  "key": "analysis"
                },
                {
                  "label": "Data\r\n        & Forensics",
                  "link": "https://news.sky.com/data-and-forensics",
                  "key": "data-and-forensics"
                },
                {
                  "label": "Videos",
                  "link": "https://news.sky.com/videos",
                  "key": "videos"
                },
                {
                  "label": "Programmes",
                  "link": "https://news.sky.com/programmes",
                  "key": "programmes"
                },
                {
                  "label": "Weather",
                  "link": "https://news.sky.com/weather",
                  "key": "weather"
                }
              ]
            }
          },
          "navigation": {
            "menuLink": "#sdc-site-footer",
            "hide-menu-label": True,
            "items": [
              {
                "label": "Home",
                "link": "https://news.sky.com/",
                "key": "home"
              },
              {
                "label": "Election",
                "link": "https://news.sky.com/politics",
                "key": "politics",
                "current": True
              },
              {
                "label": "UK",
                "link": "https://news.sky.com/uk",
                "key": "uk"
              },
              {
                "label": "World",
                "link": "https://news.sky.com/world",
                "key": "world"
              },
              {
                "label": "Israel-Hamas\r\n        War",
                "link": "https://news.sky.com/israel-hamas-war",
                "key": "israel-hamas-war"
              },
              {
                "label": "US",
                "link": "https://news.sky.com/us",
                "key": "us"
              },
              {
                "label": "Climate",
                "link": "https://news.sky.com/climate",
                "key": "climate"
              },
              {
                "label": "Science & Tech",
                "link": "https://news.sky.com/technology",
                "key": "tech"
              },
              {
                "label": "Business",
                "link": "https://news.sky.com/business",
                "key": "business"
              },
              {
                "label": "Ents\r\n        & Arts",
                "link": "https://news.sky.com/entertainment",
                "key": "entertainment"
              },
              {
                "label": "Offbeat",
                "link": "https://news.sky.com/strangenews",
                "key": "strangenews"
              },
              {
                "label": "Travel",
                "link": "https://news.sky.com/travel",
                "key": "travel"
              },
              {
                "label": "Analysis",
                "link": "https://news.sky.com/analysis",
                "key": "analysis"
              },
              {
                "label": "Data\r\n        & Forensics",
                "link": "https://news.sky.com/data-and-forensics",
                "key": "data-and-forensics"
              },
              {
                "label": "Videos",
                "link": "https://news.sky.com/videos",
                "key": "videos"
              },
              {
                "label": "Programmes",
                "link": "https://news.sky.com/programmes",
                "key": "programmes"
              },
              {
                "label": "Weather",
                "link": "https://news.sky.com/weather",
                "key": "weather"
              }
            ],
            "search": False,
            "overflow": {
              "auto": True,
              "label": "More",
              "link": "#",
              "items": True
            },
            "action-button": {
              "link": "https://news.sky.com/watch-live",
              "label": "Watch\r\n        Live",
              "top": True,
              "mobile-header": True
            },
            "mobile-action-button": False
          }
        }
      },
      "votingAreaId": "2024-07-04-UK_GENERAL_PA_1",
      "election": {
        "name": "General Election",
        "electionType": "UK_GENERAL",
        "metaData": {
          "title": "General Election 2024",
          "description": "Get\r\n        real time results and latest news on the UK general election. Find out who won and lost in your area and see the breakdown of vote share.",
          "image": "share-image-general-election-2024.png"
        },
        "id": "2024-07-04-UK_GENERAL",
        "slug": "general-election-2024",
        "year": "2024",
        "mainParties": [
          "Lab",
          "C",
          "LD",
          "SNP",
          "Green",
          "Reform",
          "PC"
        ],
        "mapKey": [
          {
            "label": "Con",
            "party": "c"
          },
          {
            "label": "Lab",
            "party": "lab"
          },
          {
            "label": "SNP",
            "party": "snp"
          },
          {
            "label": "Lib\r\n        Dem",
            "party": "lib-dem"
          },
          {
            "label": "Plaid Cymru",
            "party": "pc"
          },
          {
            "label": "Green",
            "party": "green"
          },
          {
            "label": "Reform",
            "party": "reform"
          },
          {
            "label": "DUP",
            "party": "dup"
          },
          {
            "label": "SF",
            "party": "sf"
          },
          {
            "label": "Alliance",
            "party": "alliance"
          },
          {
            "label": "SDLP",
            "party": "sdlp"
          },
          {
            "label": "UUP",
            "party": "uup"
          }
        ]
      },
      "fallbackData": {
        "votingArea": {
          "shortName": "Aberafan\r\n        Maesteg",
          "votingStatus": "DECLARED",
          "electorate": 72467,
          "turnoutPercentage": 49.339699449404556,
          "nominations": [
            {
              "candidate": {
                "firstName": "Stephen",
                "surname": "Kinnock"
              },
              "party": {
                "abbreviation": "Lab",
                "name": "Labour",
                "displayName": "Lab"
              },
              "voteCount": 17838,
              "votePercentage": 49.889525940427916,
              "votePercentageChange": -3.031258337986415
            },
            {
              "candidate": {
                "firstName": "Mark",
                "surname": "Griffiths"
              },
              "party": {
                "abbreviation": "Reform",
                "name": "Reform\r\n        UK",
                "displayName": "Reform"
              },
              "voteCount": 7484,
              "votePercentage": 20.931338274367224,
              "votePercentageChange": 12.390717424807312
            },
            {
              "candidate": {
                "firstName": "Colin",
                "surname": "Deere"
              },
              "party": {
                "abbreviation": "PC",
                "name": "Plaid Cymru",
                "displayName": "PC"
              },
              "voteCount": 4719,
              "votePercentage": 13.198154104321075,
              "votePercentageChange": 4.214069283394979
            },
            {
              "candidate": {
                "firstName": "Abigail",
                "surname": "Mainon"
              },
              "party": {
                "abbreviation": "C",
                "name": "Conservative",
                "displayName": "Con"
              },
              "voteCount": 2903,
              "votePercentage": 8.119144175639772,
              "votePercentageChange": -14.508773794780957
            },
            {
              "candidate": {
                "firstName": "Nigel",
                "surname": "Hill"
              },
              "party": {
                "abbreviation": "Green",
                "name": "Green",
                "displayName": "Green"
              },
              "voteCount": 1094,
              "votePercentage": 3.0597119284016223,
              "votePercentageChange": 1.481700537905708
            },
            {
              "candidate": {
                "firstName": "Justin",
                "surname": "Griffiths"
              },
              "party": {
                "abbreviation": "LD",
                "name": "Liberal\r\n        Democrat",
                "displayName": "Lib Dem"
              },
              "voteCount": 916,
              "votePercentage": 2.561879457418543,
              "votePercentageChange": -1.1411572577965483
            },
            {
              "candidate": {
                "firstName": "Captain",
                "surname": "Beany"
              },
              "party": {
                "abbreviation": "Ind",
                "name": "Independent",
                "displayName": "Ind"
              },
              "voteCount": 618,
              "votePercentage": 1.7284295902670954,
              "votePercentageChange": 1.7284295902670954
            },
            {
              "candidate": {
                "firstName": "Rhiannon",
                "surname": "Morrissey"
              },
              "party": {
                "abbreviation": "Heritage",
                "name": "Heritage\r\n        Party",
                "displayName": "Heritage"
              },
              "voteCount": 183,
              "votePercentage": 0.5118165291567613,
              "votePercentageChange": 0.5118165291567613
            }
          ],
          "gainOrHold": "HOLD",
          "majority": 10354,
          "majorityShare": 28.95818766606069,
          "electedCandidate": {
            "party": {
              "name": "Labour",
              "abbreviation": "Lab",
              "displayName": "Lab"
            }
          },
          "previouslyElectedCandidate": {
            "party": {
              "displayName": "Lab",
              "name": "Labour"
            }
          }
        }
      },
      "metaData": {
        "title": "Aberafan\r\n        Maesteg | General Election 2024",
        "description": "Get real time results and latest news on the UK general election. Find out who won and lost in your area and see the breakdown of vote share.",
        "image": "share-image-general-election-2024.png",
        "url": "https://election.news.sky.com/elections/general-election-2024/aberafan-maesteg-1"
      }
    },
    "__N_SSP": True
  },
  "page": "/elections/[election]/[constituency]",
  "query": {
    "election": "general-election-2024",
    "constituency": "aberafan-maesteg-1"
  },
  "buildId": "W56tnVzjPPoRnC3lRU45j",
  "isFallback": False,
  "isExperimentalCompile": False,
  "gssp": True,
  "scriptLoader": []
}

In [31]:
data['props']['pageProps']['components']['analytics']['name']

'Aberafan Maesteg'

In [47]:
data

{'props': {'pageProps': {'components': {'consentManagement': {'enabled': True,
     'accountId': 630,
     'propertyId': 9804,
     'privacyManagerId': 617780,
     'spv2': True,
     'propertyHref': 'https://election.news.sky.com/',
     'joinHref': 'False'},
    'chartbeat': {'enabled': 'True',
     'uid': '34974',
     'domain': 'news.sky.com',
     'pageTitle': 'Aberafan\r\n        Maesteg | General Election 2024'},
    'analytics': {'applicationEnvironment': 'production',
     'analyticsEnvironment': 'prod',
     'application': 'Elections',
     'name': 'Aberafan Maesteg',
     'breadcrumb': ['Election',
      'General Election 2024',
      'Area',
      'Aberafan Maesteg'],
     'type': 'area',
     'tags': {'name': 'section', 'value': 'Election:Area'},
     'pageTitle': 'Aberafan\r\n        Maesteg | General Election 2024'},
    'launch': {'appId': 'ENdec3197734dd4b629a9283183faf094b'},
    'electionNav': {'navigation': [{'label': 'Vote 2022', 'link': '/'},
      {'label': 'Nort

In [57]:
data['props']['pageProps']['fallbackData']['votingArea']['electedCandidate']

{'shortName': 'Aberafan\r\n        Maesteg',
 'votingStatus': 'DECLARED',
 'electorate': 72467,
 'turnoutPercentage': 49.339699449404556,
 'nominations': [{'candidate': {'firstName': 'Stephen', 'surname': 'Kinnock'},
   'party': {'abbreviation': 'Lab', 'name': 'Labour', 'displayName': 'Lab'},
   'voteCount': 17838,
   'votePercentage': 49.889525940427916,
   'votePercentageChange': -3.031258337986415},
  {'candidate': {'firstName': 'Mark', 'surname': 'Griffiths'},
   'party': {'abbreviation': 'Reform',
    'name': 'Reform\r\n        UK',
    'displayName': 'Reform'},
   'voteCount': 7484,
   'votePercentage': 20.931338274367224,
   'votePercentageChange': 12.390717424807312},
  {'candidate': {'firstName': 'Colin', 'surname': 'Deere'},
   'party': {'abbreviation': 'PC', 'name': 'Plaid Cymru', 'displayName': 'PC'},
   'voteCount': 4719,
   'votePercentage': 13.198154104321075,
   'votePercentageChange': 4.214069283394979},
  {'candidate': {'firstName': 'Abigail', 'surname': 'Mainon'},
  

In [17]:
df_winning['constituency_name']

0                        Aberafan Maesteg
1                          Aberdeen North
2                          Aberdeen South
3      Aberdeenshire North and Moray East
4                      Airdrie and Shotts
                      ...                
644             Wythenshawe and Sale East
645                                Yeovil
646                              Ynys Mon
647                          York Central
648                            York Outer
Name: constituency_name, Length: 649, dtype: object

In [54]:
json_data=data['props']['pageProps']['fallbackData']['votingArea']['nominations']

In [35]:
data['props']['pageProps']['components']['analytics']['name']

'Aberafan Maesteg'

'Labour'

In [62]:
df = pd.json_normalize(json_data, sep='_')
df['constituency'] = data['props']['pageProps']['components']['analytics']['name']
df['party_name'] = df['party_name'].str.replace('\r\n', '')
df['Electorate_size']=data['props']['pageProps']['fallbackData']['votingArea']['electorate']
df['Turnout_perc']=data['props']['pageProps']['fallbackData']['votingArea']['turnoutPercentage']
df['new_party']=data['props']['pageProps']['fallbackData']['votingArea']['electedCandidate']['party']['name']
df['old_party']=data['props']['pageProps']['fallbackData']['votingArea']['previouslyElectedCandidate']['party']['name']

In [63]:
df

,voteCount,votePercentage,votePercentageChange,candidate_firstName,candidate_surname,party_abbreviation,party_name,party_displayName,constituency,Electorate_size,Turnout_perc,new_party,old_party
0,17838,49.889526,-3.031258,Stephen,Kinnock,Lab,Labour,Lab,Aberafan Maesteg,72467,49.339699,Labour,Labour
1,7484,20.931338,12.390717,Mark,Griffiths,Reform,Reform UK,Reform,Aberafan Maesteg,72467,49.339699,Labour,Labour
2,4719,13.198154,4.214069,Colin,Deere,PC,Plaid Cymru,PC,Aberafan Maesteg,72467,49.339699,Labour,Labour
3,2903,8.119144,-14.508774,Abigail,Mainon,C,Conservative,Con,Aberafan Maesteg,72467,49.339699,Labour,Labour
4,1094,3.059712,1.481701,Nigel,Hill,Green,Green,Green,Aberafan Maesteg,72467,49.339699,Labour,Labour
5,916,2.561879,-1.141157,Justin,Griffiths,LD,Liberal Democrat,Lib Dem,Aberafan Maesteg,72467,49.339699,Labour,Labour
6,618,1.728430,1.728430,Captain,Beany,Ind,Independent,Ind,Aberafan Maesteg,72467,49.339699,Labour,Labour
7,183,0.511817,0.511817,Rhiannon,Morrissey,Heritage,Heritage Party,Heritage,Aberafan Maesteg,72467,49.339699,Labour,Labour


In [64]:
base_link='https://election.news.sky.com/elections/general-election-2024/'

In [81]:
import requests
from bs4 import BeautifulSoup

# URL to fetch HTML from
url = base_link+'1'
url

'https://election.news.sky.com/elections/general-election-2024/1'

In [82]:
import json
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    html_content = response.content
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the script tag with id="__NEXT_DATA__"
    script_tag = soup.find('script', id='__NEXT_DATA__')
    
    if script_tag:
        # Extract the JSON data from the script tag
        json_data = script_tag.string
        
        # Convert the JSON string to a Python dictionary
        data_dict = json.loads(json_data)
        
        # Print the JSON data (or process it as needed)
        print(data_dict)
    else:
        print(f"Script tag with id='__NEXT_DATA__' not found on {url}")
else:
    print(f"Failed to retrieve data from URL: {url}. Status code: {response.status_code}")

{'props': {'pageProps': {'components': {'consentManagement': {'enabled': True, 'accountId': 630, 'propertyId': 9804, 'privacyManagerId': 617780, 'spv2': True, 'propertyHref': 'https://election.news.sky.com/', 'joinHref': 'false'}, 'chartbeat': {'enabled': 'true', 'uid': '34974', 'domain': 'news.sky.com', 'pageTitle': 'Aberafan Maesteg | General Election 2024'}, 'analytics': {'applicationEnvironment': 'production', 'analyticsEnvironment': 'prod', 'application': 'Elections', 'name': 'Aberafan Maesteg', 'breadcrumb': ['Election', 'General Election 2024', 'Area', 'Aberafan Maesteg'], 'type': 'area', 'tags': {'name': 'section', 'value': 'Election:Area'}, 'pageTitle': 'Aberafan Maesteg | General Election 2024'}, 'launch': {'appId': 'ENdec3197734dd4b629a9283183faf094b'}, 'electionNav': {'navigation': [{'label': 'Vote 2022', 'link': '/'}, {'label': 'Northern Ireland Assembly', 'link': '/elections/northern-ireland-2022', 'current': True}]}, 'footerData': {'properties': {'logo': '/resources/sky-

In [74]:
df = pd.json_normalize(data_dict, sep='_')

In [75]:
df

,page,buildId,isFallback,isExperimentalCompile,gssp,scriptLoader,props_pageProps_components_consentManagement_enabled,props_pageProps_components_consentManagement_accountId,props_pageProps_components_consentManagement_propertyId,props_pageProps_components_consentManagement_privacyManagerId,...,props_pageProps_fallbackData_votingArea_electedCandidate_party_displayName,props_pageProps_fallbackData_votingArea_previouslyElectedCandidate_party_displayName,props_pageProps_fallbackData_votingArea_previouslyElectedCandidate_party_name,props_pageProps_metaData_title,props_pageProps_metaData_description,props_pageProps_metaData_image,props_pageProps_metaData_url,props___N_SSP,query_election,query_constituency
0,/elections/[election]/[constituency],W56tnVzjPPoRnC3lRU45j,False,False,True,[],True,630,9804,617780,...,SNP,SNP,Scottish National Party,Aberdeen North | General Election 2024,Get real time results and latest news on the U...,share-image-general-election-2024.png,https://election.news.sky.com/elections/genera...,True,general-election-2024,2


In [83]:
def json_to_dataframe(data):
    json_data=data['props']['pageProps']['fallbackData']['votingArea']['nominations']
    # Normalize JSON data into a DataFrame
    df = pd.json_normalize(json_data, sep='_')
    
    # Extract values and assign to DataFrame columns
    df['constituency'] = data['props']['pageProps']['components']['analytics']['name']
    df['party_name'] = df['party_name'].str.replace('\r\n', '')
    df['Electorate_size'] = data['props']['pageProps']['fallbackData']['votingArea']['electorate']
    df['Turnout_perc'] = data['props']['pageProps']['fallbackData']['votingArea']['turnoutPercentage']
    df['new_party'] = data['props']['pageProps']['fallbackData']['votingArea']['electedCandidate']['party']['name']
    df['old_party'] = data['props']['pageProps']['fallbackData']['votingArea']['previouslyElectedCandidate']['party']['name']
    
    return df

In [84]:
ans2=json_to_dataframe(data_dict)

In [85]:
ans2

,voteCount,votePercentage,votePercentageChange,candidate_firstName,candidate_surname,party_abbreviation,party_name,party_displayName,constituency,Electorate_size,Turnout_perc,new_party,old_party
0,17838,49.889526,-3.031258,Stephen,Kinnock,Lab,Labour,Lab,Aberafan Maesteg,72467,49.339699,Labour,Labour
1,7484,20.931338,12.390717,Mark,Griffiths,Reform,Reform UK,Reform,Aberafan Maesteg,72467,49.339699,Labour,Labour
2,4719,13.198154,4.214069,Colin,Deere,PC,Plaid Cymru,PC,Aberafan Maesteg,72467,49.339699,Labour,Labour
3,2903,8.119144,-14.508774,Abigail,Mainon,C,Conservative,Con,Aberafan Maesteg,72467,49.339699,Labour,Labour
4,1094,3.059712,1.481701,Nigel,Hill,Green,Green,Green,Aberafan Maesteg,72467,49.339699,Labour,Labour
5,916,2.561879,-1.141157,Justin,Griffiths,LD,Liberal Democrat,Lib Dem,Aberafan Maesteg,72467,49.339699,Labour,Labour
6,618,1.728430,1.728430,Captain,Beany,Ind,Independent,Ind,Aberafan Maesteg,72467,49.339699,Labour,Labour
7,183,0.511817,0.511817,Rhiannon,Morrissey,Heritage,Heritage Party,Heritage,Aberafan Maesteg,72467,49.339699,Labour,Labour


In [80]:
ans

,voteCount,votePercentage,votePercentageChange,candidate_firstName,candidate_surname,party_abbreviation,party_name,party_displayName,constituency,Electorate_size,Turnout_perc,new_party,old_party
0,14533,34.524290,-18.373350,Kirsty,Blackman,SNP,Scottish National Party,SNP,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
1,12773,30.343271,18.363699,Lynn,Thomson,Lab,Labour,Lab,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
2,5881,13.970780,-10.578864,Gillian,Tebberen,C,Conservative,Con,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
3,3781,8.982064,7.360184,Kenneth,Leggat,Reform,Reform UK,Reform,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
4,2583,6.136121,-1.594244,Desmond,Bouse,LD,Liberal Democrat,Lib Dem,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
5,1275,3.028863,1.807964,Esme,Houston,Green,Green,Green,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
6,703,1.670032,1.670032,Charlie,Abel,Alba,Alba,Alba,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
7,352,0.836204,0.836204,Dawn,Smith,Scot Family,Scottish Family Party,Scot Fam,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party
8,214,0.508374,0.508374,Lucas,Grant,TUSC,Trade Unionist and Socialist Coalition,TUSC,Aberdeen North,75925,55.442871,Scottish National Party,Scottish National Party


In [90]:
an=ans.merge(ans2,how='outer')

In [137]:
columns = ['voteCount', 'votePercentage', 'votePercentageChange',
           'candidate_firstName', 'candidate_surname', 'party_abbreviation',
           'party_name', 'party_displayName', 'constituency', 'Electorate_size',
           'Turnout_perc', 'new_party', 'old_party']

# Create an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)

In [138]:
for i in range(1, 651):
    url = base_link + str(i)

    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content
        html_content = response.content
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the script tag with id="__NEXT_DATA__"
        script_tag = soup.find('script', id='__NEXT_DATA__')
        
        if script_tag:
            # Extract the JSON data from the script tag
            json_data = script_tag.string
            
            # Convert the JSON string to a Python dictionary
            data_dict = json.loads(json_data)
            
            # Assuming json_to_dataframe function converts JSON data to DataFrame
            temp = json_to_dataframe(data_dict)
            
            # Merge temp with df using outer join and update df
            df = pd.merge(df, temp, how='outer')
        else:
            print(f"Script tag with id='__NEXT_DATA__' not found on {url}")
    else:
        print(f"Failed to retrieve data from URL: {url}. Status code: {response.status_code}")

# Now df should contain merged data from all pages


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4515 entries, 0 to 4514
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   voteCount             4515 non-null   int64  
 1   votePercentage        4515 non-null   float64
 2   votePercentageChange  4515 non-null   float64
 3   candidate_firstName   4515 non-null   object 
 4   candidate_surname     4515 non-null   object 
 5   party_abbreviation    4515 non-null   object 
 6   party_name            4515 non-null   object 
 7   party_displayName     4515 non-null   object 
 8   constituency          4515 non-null   object 
 9   Electorate_size       4515 non-null   int64  
 10  Turnout_perc          4515 non-null   float64
 11  new_party             4515 non-null   object 
 12  old_party             4515 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 458.7+ KB


In [140]:
df['constituency'].unique()

array(['Aberafan Maesteg', 'Aberdeen North', 'Aberdeen South',
       'Aberdeenshire N & Moray E', 'Aberdeenshire W & Kincardine',
       'Airdrie & Shotts', 'Aldershot', 'Aldridge-Brownhills',
       'Alloa & Grangemouth', 'Altrincham & Sale West',
       'Alyn and Deeside', 'Amber Valley', 'Angus & Perthshire Glens',
       'Antrim East', 'Antrim North', 'Antrim South',
       'Arbroath & Broughty Ferry', 'Argyll, Bute & S Lochaber',
       'Arundel & South Downs', 'Ashfield', 'Ashford',
       'Ashton-under-Lyne', 'Aylesbury', 'Ayr, Carrick & Cumnock',
       'Ayrshire Central', 'Ayrshire North & Arran', 'Banbury',
       'Bangor Aberconwy', 'Barking', 'Barnsley North', 'Barnsley South',
       'Barrow & Furness', 'Basildon & Billericay',
       'Basildon South & E Thurrock', 'Basingstoke', 'Bassetlaw', 'Bath',
       'Bathgate & Linlithgow', 'Battersea', 'Beaconsfield',
       'Beckenham & Penge', 'Bedford', 'Bedfordshire Mid',
       'Bedfordshire North', 'Belfast East', 'Belfast 

In [141]:
df_winning['constituency_name'].nunique()

649

In [142]:
df.sample(5)

,voteCount,votePercentage,votePercentageChange,candidate_firstName,candidate_surname,party_abbreviation,party_name,party_displayName,constituency,Electorate_size,Turnout_perc,new_party,old_party
2507,3826,9.145015,5.808735,Sharmen,Rahman,Green,Green,Green,Leicester South,70867,59.035941,Independent,Labour
1193,2151,4.557493,2.662920,Te Ata,Browne,Green,Green,Green,Crewe & Nantwich,78423,60.182599,Labour,Conservative
3068,1336,3.816380,0.063795,David,Schmitz,LD,Liberal Democrat,Lib Dem,Nottingham North & Kimberley,73768,47.455536,Labour,Labour
1934,1036,2.829132,-2.094683,John,Lawson,LD,Liberal Democrat,Lib Dem,Great Grimsby & Cleethorpes,77050,47.526282,Labour,Conservative
3847,650,1.519757,1.519757,Alexander,Bramham,Soc Dem,Social Democratic Party,SDP,"Stone, Gt Wyrley & Penkridge",69378,61.647785,Conservative,Conservative


In [143]:
df.columns

Index(['voteCount', 'votePercentage', 'votePercentageChange',
       'candidate_firstName', 'candidate_surname', 'party_abbreviation',
       'party_name', 'party_displayName', 'constituency', 'Electorate_size',
       'Turnout_perc', 'new_party', 'old_party'],
      dtype='object')

In [136]:
new_order = ['constituency', 'candidate_firstName', 'candidate_surname', 'party_abbreviation','party_name','voteCount', 'votePercentage', 'votePercentageChange', 'Electorate_size', 'Turnout_perc', 'new_party', 'old_party']
df = df[new_order]  

KeyError: "['party_name'] not in index"

In [134]:
df

In [124]:
df.to_csv('voting_data.csv', index=False)

In [93]:
base_link+str(56)

'https://election.news.sky.com/elections/general-election-2024/56'